# 訓練資料前處理

### 第一階段
- 去除髒話
- 翻譯

In [1]:
from better_profanity import profanity
from googletrans import Translator
import csv

### 引入翻譯以及髒話清理套件

profanity 可以篩選出許多常見英文髒話，也會偵測類似字 (EX : ass, 4ss, bitch, b1tch)

In [ ]:
translator = Translator()
datalist = []

with open('abusive.csv', newline='', encoding='utf-8') as csvfile:
    rows = csv.reader(csvfile)
    for row in rows:
        #清除髒字
        text = profanity.censor(row, "")
        try:
            result = translator.translate(text, dest='zh-tw').text
            datalist.append(result)
        except:
            continue

### 存成 CSV
#### stage1.csv

In [ ]:
import pandas as pd
bar_name = list(datalist) 
store = pd.DataFrame(data = datalist)
store.to_csv('abusive_clean_stage1.csv', encoding = 'utf-8_sig',index = True)

### 第二階段
- 清除其餘髒東西 (URL, RT, @...)
####  Regex 邏輯：

##### 照順序往下掃

- 將用戶 @blablabla 替換成 "你"
- 將網址去除
- 空格去除
- 非中文去除 (除了英文，還可以去除 emoji 和 # 之類的麻煩符號)

In [ ]:
import re

user = re.compile('(@[^\w]*)')
url = re.compile('((https?):((//)|(\\\\))+[\w\d:#@%/;$()~_?\+-=\\\.&]*)')
spacechar = re.compile('(\\s){1,}')
non_chinese = re.compile('([^\u4e00-\u9fa5，。！：])')

### 開始清理
##### 將 stage1.csv 餵進去二次清洗

In [ ]:
datalist = []

with open('abusive_clean_stage1.csv', newline='', encoding='utf-8') as csvfile:
  rows = csv.reader(csvfile)
  for row in rows:
        content = re.sub(url, "", row[1])
        content = re.sub(user, "你", content)
        content = re.sub("RT", "", content)
        content = re.sub(spacechar, "", content)
        content = re.sub(non_chinese, "", content)
        content = re.sub("你：", "", content)
        content = re.sub("白痴", "人", content)
        #去除所有疊字
        content = re.sub(r"(.)(\1+)", r"\1", content)
        if content != "":
            datalist.append(content)

#### 確認一下資料有沒有清乾淨

In [ ]:
datalist

### 存成 CSV
#### stage2.csv

In [ ]:
import pandas as pd
bar_name = list(datalist) 
store = pd.DataFrame(data = datalist)
store.to_csv('abusive_clean_stage2.csv', encoding = 'utf-8_sig',index = True)